In [1]:
# Jupyter notebook related
%reload_ext autoreload
%autoreload 2

In [2]:
import hvplot.xarray  # noqa
import hvplot.pandas  # noqa
import panel as pn  # noqa
import panel.widgets as pnw

In [3]:
import os
import geopandas as gpd
from dask import delayed

# from elogs import Elogs, ElogsTask

with open('../../../connstr_vegteam') as f:
    connect_str = f.read()
container_name = 'evotrain'

locs_fn = "../../../locations_v2.csv"

In [4]:
import xarray as xr


In [5]:
fn = 'evotrain_v2ts_2020_18RTN_099_34_20m.nc'

darr = xr.open_dataarray(fn)

In [7]:
darr

<xarray.DataArray 'evotrain_v2ts_2020_18RTN_099_34_20m' (time: 62, band: 7,
                                                         y: 64, x: 64)>
[1777664 values with dtype=uint16]
Coordinates: (12/43)
  * time                                     (time) datetime64[ns] 2020-01-03...
    id                                       (time) object ...
  * band                                     (band) object 'B05' 'B06' ... 'SCL'
  * x                                        (x) float64 2.026e+05 ... 2.038e+05
  * y                                        (y) float64 2.703e+06 ... 2.701e+06
    s2:datatake_id                           (time) object ...
    ...                                       ...
    title                                    (band) object ...
    gsd                                      float64 ...
    common_name                              (band) object ...
    center_wavelength                        (band) float64 ...
    full_width_half_max                      (band) float64 ...
    epsg                                     object ...
Attributes:
    spec:        RasterSpec(epsg='EPSG:32618', bounds=(202540, 2701460, 20382...
    crs:         epsg:EPSG:32618
    transform:   [ 2.00000e+01  0.00000e+00  2.02540e+05  0.00000e+00 -2.0000...
    resolution:  20
    bounds:      [ 202540. 2701460.  203820. 2702740.]

# Training data extraction

In [4]:
max_cloud_cover = 90
max_workers = 10
year = 2020

In [5]:
import time
from pathlib import Path


def store_ts(ts, fn, complevel=5):
    if 'spec' in ts.attrs.keys():
        ts.attrs['spec'] = str(ts.attrs['spec'])
    for var in ['proj:bbox', 'proj:shape', 'proj:transform']:
        if var in ts.coords:
            ts = ts.drop_vars(var)
    encoding = {ts.name: {'zlib': True, 'complevel': complevel}}
    # Save to a NetCDF file with compression
    ts.to_netcdf(fn, encoding=encoding)


In [6]:
def _extract_loc(patch_id, tile, epsg, xmin, ymin, xmax, ymax, year):
    from loguru import logger
    from satio_pc.reader import S2TileReader
    from satio_pc.preprocessing.clouds import preprocess_scl
    from satio_pc.sentinel2 import BANDS_RESOLUTION
    from satio_pc.geotiff import slash_tile
    from satio_pc.utils.azure import AzureBlobReader

    from rasterio.enums import Resampling
    
    container_name = 'evotrain'
    azure = AzureBlobReader(connect_str,
                            container_name)

    bounds = xmin, ymin, xmax, ymax
    
    
    fns = [f'v2ts/{year}/{slash_tile(tile)}/{pre}/evotrain_v2ts_{year}_{patch_id}_{pre}.nc'
           for pre in '10m 20m 60m'.split()]
    
    if all(map(lambda fn: azure.check_file_exists(fn), fns)):
        logger.warning(f"Targets {fns} exists, skipping...")
        return True

    start_date = f'{year}-01-01'
    end_date = f'{year + 1}-01-01'

    reader = S2TileReader(tile,
                          start_date,
                          end_date,
                          max_cloud_cover)

    logger.info("Loading SCL mask")
    scl = reader.read(bounds, epsg, ['SCL'], max_workers=max_workers)
    
    logger.info("Filtering no data obs")
    valid_obs = scl.mean(dim=('y', 'x', 'band')) > 0

    scl = scl.sel(time=valid_obs)
    
    reader._items = [i for i, b in zip(reader._items, valid_obs) if b]

    bands = ['B01', 'B09',
             'B02', 'B03', 'B04', 'B08', 
             'B05', 'B06', 'B07', 'B8A', 'B11', 'B12']

    bands_10m = [b for b in bands if BANDS_RESOLUTION[b] == 10]
    bands_20m = [b for b in bands if BANDS_RESOLUTION[b] == 20] + ['SCL']
    bands_60m = [b for b in bands if BANDS_RESOLUTION[b] == 60] + ['AOT', 'WVP']
    
    logger.info("Loading data")
    ts10 = reader.read(bounds, epsg, bands_10m, max_workers=20,
                      resolution=10).ewc.harmonize()
    ts20 = reader.read(bounds, epsg, bands_20m, max_workers=20,
                       resolution=20).ewc.harmonize()
    ts60 = reader.read(bounds, epsg, bands_60m, max_workers=20,
                       resolution=60).ewc.harmonize()
    
    for ts, pre in zip([ts10, ts20, ts60],
                       ['10m', '20m', '60m']):
        fn = f'evotrain_v2ts_{year}_{patch_id}_{pre}.nc'
        dst_fn = f"v2ts/{year}/{slash_tile(tile)}/{pre}/{fn}"
        
        logger.info(f"Saving {fn} and uploading to {dst_fn}")
        ts.name = f'evotrain_v2ts_{year}_{patch_id}_{pre}'
        store_ts(ts, fn, complevel=4)
    
        azure.upload_file(fn,
                          dst_fn,
                          overwrite=True)
        os.remove(fn)
    
    logger.success("Done")
    return True

In [7]:
def extract_loc(tup):
    patch_id, tile, epsg, xmin, ymin, xmax, ymax, year = tup
    xmin, ymin, xmax, ymax = list(map(float, (xmin, ymin, xmax, ymax)))
    epsg = int(epsg)
    year = int(year)
    
    try:
        from loguru import logger
        return _extract_loc(patch_id, tile, epsg, xmin, ymin, xmax, ymax, year)
    except Exception as e:
        logger.exception(e)
        return False

In [8]:
import pandas as pd
import xarray as xr
import numpy as np

def split_dataframe(df, n, nth_id):
    if nth_id >= n or nth_id < 0:
        raise ValueError("Invalid nth_id value")
    chunk_size = len(df) // n
    remainder = len(df) % n
    start_idx = nth_id * chunk_size + min(nth_id, remainder)
    end_idx = start_idx + chunk_size + (1 if nth_id < remainder else 0)
    return df.iloc[start_idx:end_idx]


locs = pd.read_csv(locs_fn)
locs = locs.sample(frac=1, random_state=0)  # shuffle

# chunk_id = 0
# locs10 = split_dataframe(locs, 10, chunk_id)

# locs = locs[~locs.patch_id.isin(locs10.patch_id.values)]

cols = ['patch_id', 'tile', 'epsg', 'xmin', 'ymin', 'xmax', 'ymax']
locs = locs[cols]

In [9]:
locs.shape

(177341, 7)

In [26]:
from satio_pc.utils.azure import AzureBlobReader
azure = AzureBlobReader(connect_str,
                        container_name)

# keys = azure.list_files()
def list_files(prefix=None):
    files = set()
    for blob in azure.container_client.list_blobs(name_starts_with=prefix):
        if blob.name[-1] != "/":
            files.add(blob.name)
    return files

keys = list_files(f'v2ts/{year}/')

splits = list(map(lambda key: key.split('.')[0].split('_')[-4:], keys))

ids =  list(map(lambda s: "_".join(s[:-1]), splits))
res =  list(map(lambda s: s[-1], splits))

df = pd.DataFrame(zip(ids, res), columns=['ids', 'res'])

print((df.value_counts('ids') == 3).sum(), (df.value_counts('ids') < 3).sum())

done_ids = (df.value_counts('ids') == 3).index.values

2023-10-04 15:59:30,329 - distributed.client - ERROR - Failed to reconnect to scheduler after 30.00 seconds, closing client


176962 0


In [27]:
locs = locs[~locs.patch_id.isin(done_ids)]
locs.shape

(379, 7)

In [11]:
locs.to_parquet('locs.parq', index=False)

# Cluster setup

In [14]:
# from dask_gateway import Gateway
# gateway = Gateway()


# # List the clusters and get the cluster report
# clusters_reports = gateway.list_clusters()

# # Get the first cluster report
# cluster_report = clusters_reports[0]

# # Connect to the cluster using the cluster report
# cluster = gateway.connect(cluster_report)

# # Get the client object from the cluster
# client = cluster.get_client()

In [25]:
# stop clusters
from dask_gateway import Gateway
gateway = Gateway()
clusters_reports = gateway.list_clusters()

clusters = [gateway.stop_cluster(c.name) for c in clusters_reports]
clusters

[]

In [16]:
# create and scale cluster
from dask.distributed import PipInstall, Client
import dask_gateway

cluster = dask_gateway.GatewayCluster()
client = cluster.get_client()

print(client.dashboard_link)

cluster.scale(100)

cluster

https://pccompute.westeurope.cloudapp.azure.com/compute/services/dask-gateway/clusters/prod.1a9bade995b54b9e9122121ba9a8a47f/status


In [17]:
# Once cluster is scaled, install satio_pc
# satio_pc_url = "https://s3-eu-central-1.amazonaws.com/vito-worldcover-public/wheels/satio_pc-0.0.1-py3-none-any.whl"
satio_pc_url = "satio-pc==0.0.3"
plugin = PipInstall(packages=[satio_pc_url])
client.register_worker_plugin(plugin)

{'tls://10.244.0.144:43445': {'status': 'OK'},
 'tls://10.244.0.145:36127': {'status': 'OK'},
 'tls://10.244.0.146:33891': {'status': 'OK'},
 'tls://10.244.114.186:39355': {'status': 'OK'},
 'tls://10.244.114.187:36557': {'status': 'OK'},
 'tls://10.244.114.188:45955': {'status': 'OK'},
 'tls://10.244.117.167:41901': {'status': 'OK'},
 'tls://10.244.117.168:43093': {'status': 'OK'},
 'tls://10.244.117.169:43639': {'status': 'OK'},
 'tls://10.244.134.64:46067': {'status': 'OK'},
 'tls://10.244.134.65:41851': {'status': 'OK'},
 'tls://10.244.134.66:37871': {'status': 'OK'},
 'tls://10.244.137.182:39873': {'status': 'OK'},
 'tls://10.244.137.183:42649': {'status': 'OK'},
 'tls://10.244.137.184:40705': {'status': 'OK'},
 'tls://10.244.147.198:39703': {'status': 'OK'},
 'tls://10.244.147.199:32941': {'status': 'OK'},
 'tls://10.244.147.200:37507': {'status': 'OK'},
 'tls://10.244.157.60:38023': {'status': 'OK'},
 'tls://10.244.157.61:38835': {'status': 'OK'},
 'tls://10.244.157.62:46763': {

In [ ]:
# check logs
logs = client.get_worker_logs()

print(len(logs))

for worker, worker_logs in logs.items():
    print(f"Logs for worker {worker}:")
    for log in worker_logs:
        print(log)
    print()
    print('*'*100)

In [15]:
# shutdown cluster
# cluster.shutdown()

## Cluster processing

In [18]:
# 'patch_id', 'tile', 'epsg', 'xmin', 'ymin', 'xmax', 'ymax'
args = [(loc.patch_id,
         loc.tile,
         loc.epsg,
         loc.xmin,
         loc.ymin,
         loc.xmax,
         loc.ymax,
         year)
        for loc in locs.itertuples()
        ]
len(args)

6534

In [19]:
import numpy as np
args = np.array(args)

In [20]:
import warnings
warnings.filterwarnings("ignore")

In [21]:
# final = extract_loc(args[1000])

In [22]:
import dask

extract_delayed = dask.delayed(extract_loc)

lazy_results = [extract_delayed(ag)
                for ag in args]

In [23]:
results = dask.compute(*lazy_results)

2023-10-04 15:51:52,258 - distributed.client - WARNING - Couldn't gather 2 keys, rescheduling {'extract_loc-3252c1c4-e877-40e2-ac81-de5d2cd048fd': (), 'extract_loc-cea745e0-d6cd-4491-9d07-7ebeff9d5c80': ()}


# finish download missing

In [10]:
locs_all = pd.read_csv(locs_fn)

args = []

for year in range(2018, 2023):
    
    print(year)
    keys = list_files(f'evotrain/v2/{year}/')

    done_patch_ids = list(map(lambda key: "_".join(key.split('.')[0].split('_')[-3:]), keys))

    locs = locs_all[~locs_all.patch_id.isin(done_patch_ids)]
    
    args += [(loc.patch_id,
         loc.tile,
         loc.epsg,
         loc.xmin,
         loc.ymin,
         loc.xmax,
         loc.ymax,
         year)
        for loc in locs.itertuples()
        ]
    print(year, locs.shape)
    
args = np.array(args)

args.size

2018
2018 (199, 28)
2019
2019 (75, 28)
2020
2020 (59, 28)
2021
2021 (260, 28)
2022
2022 (741, 28)


10672

In [14]:
import dask

extract_delayed = dask.delayed(extract_loc)

lazy_results = [extract_delayed(ag)
                for ag in args]

results = dask.compute(*lazy_results)